In [1]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler
import tensorflow_datasets as tfds
import os
import contextlib
import time

In [2]:
def convert_to_example_protobuf(image, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(image).numpy()])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label.numpy()]))
    }))

def write_multiple_tfrecords(folder, prefix, dataset, n_shards):
    os.makedirs(folder, exist_ok=True)
    paths = [f'{folder}/{prefix}-{i:03d}-of-{n_shards:03d}' for i in range(n_shards)]
    with contextlib.ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path)) for path in paths]
        for index, (image, label) in enumerate(dataset):
            shard = index % n_shards
            example = convert_to_example_protobuf(image, label)
            writers[shard].write(example.SerializeToString())
    return paths

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

training_set = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=len(train_images))
validation_set = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_set = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_filepaths = write_multiple_tfrecords('fashion_mnist_tfrecords', 'train', training_set, 10)
validation_filepaths = write_multiple_tfrecords('fashion_mnist_tfrecords', 'valid', validation_set, 10)
test_filepaths = write_multiple_tfrecords('fashion_mnist_tfrecords', 'test', test_set, 10)

In [4]:
def preprocess(tfrecord):
    feature_descriptions = {
        'image': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    example = tf.io.parse_single_example(tfrecord, feature_descriptions)
    image = tf.io.parse_tensor(example['image'], out_type=tf.uint8)
    image = tf.reshape(image, shape=[28, 28, 1])
    return image, example['label']

def mnist_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None,
                  n_parse_threads=5, batch_size=32, cache=True):
    dataset = tf.data.TFRecordDataset(filepaths,
                                      num_parallel_reads=n_read_threads)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

def get_log_path():
    root_logdir = os.path.join(os.curdir, "my_logs")
    run_id = time.strftime("run_%Y_%m_%d-%H_%M")
    return os.path.join(root_logdir, run_id)

In [5]:
train_set = mnist_dataset(train_filepaths, shuffle_buffer_size=60000)
valid_set = mnist_dataset(validation_filepaths)
test_set = mnist_dataset(test_filepaths)

In [10]:
learning_rate=3e-3
input_shape=(28, 28, 1)
l2_lambda=0.01


normalization = keras.layers.Normalization(input_shape=(28, 28, 1))
normalization.adapt(train_set.map(lambda image, label: image))

model = keras.models.Sequential([
        keras.layers.InputLayer(shape=input_shape),
        normalization,

        keras.layers.Conv2D(16, (3, 3), kernel_regularizer=keras.regularizers.l2(l2_lambda), 
                            kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),

        keras.layers.Conv2D(32, (3, 3), kernel_regularizer=keras.regularizers.l2(l2_lambda), 
                            kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.3),

        keras.layers.Conv2D(64, (3, 3), kernel_regularizer=keras.regularizers.l2(l2_lambda), 
                            kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.4),

        keras.layers.Conv2D(128, (3, 3), kernel_regularizer=keras.regularizers.l2(l2_lambda),
                            kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("elu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        
        keras.layers.Dense(10, activation="softmax", kernel_regularizer=keras.regularizers.l2(l2_lambda))
    ])

optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [8]:
tensorboard_cb = keras.callbacks.TensorBoard(get_log_path())
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

# model.fit(train_set, epochs=100, validation_data=valid_set, callbacks=[tensorboard_cb, early_stopping_cb, lr_scheduler])

# model.save("fashion_mnist_model.h5")
# model.save("fashion_mnist_model.keras")

In [13]:
model = tf.keras.models.load_model("fashion_mnist_model.keras", custom_objects={"l2_lambda": l2_lambda})

model.evaluate(test_set)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9092 - loss: 0.3291


[0.3311062157154083, 0.9053000211715698]